# **Supervised Learning**

## Metaverse Financial Transactions

<img src="../images/ia.jpg" alt="Inteligência Artificial">

## **Introduction**

### **About Dataset**

The Metaverse Financial Transactions Dataset offers a comprehensive collection of blockchain-based financial activities within the Open Metaverse. With 78,600 transactions, it provides detailed attributes like timestamps, addresses, transaction types, and risk scores. This dataset supports research in fraud detection, risk assessment, and user behavior analysis in virtual environments. It's curated to reflect the complexity of blockchain activities, aiding the development of secure digital asset management. 

### **About the Problem**

The objective of this project is to develop predictive models for anomaly detection, fraud analysis, and risk assessment in financial transactions within the Metaverse. This constitutes a multivariate classification problem, where the aim is to categorize transactions into different risk levels based on attributes such as transaction type, user behavior, and transaction amount. The target variable is the risk level, which can be categorized as high risk, moderate risk, or low risk.

### **About the Solution**

The solution to this problem involves employing supervised learning techniques on the provided Metaverse Financial Transactions Dataset. A portion of the dataset will be designated as the training set, utilized to train the model, while another portion will serve as the test set for model evaluation.

The chosen supervised learning algorithm will undergo training on the training set, learning patterns and relationships between various attributes of Metaverse financial transactions. Subsequently, the trained model will be evaluated using the test set to assess its performance and generalization ability.

The primary evaluation metric for this solution will be accuracy, representing the percentage of correctly classified transactions by the model. Achieving high accuracy is crucial for ensuring the effectiveness of the model in accurately categorizing transactions into their respective risk levels.

---

This project was made by:

| Name | UP |
|-|-|
| Ana Carolina Coutinho | up202108685 |
| José Costa | upxxxx |
| Afonso Poças | upxxxx |
